### Fase 2 - Punto 5

In [1]:
from sys import maxsize
from itertools import permutations
from math import sqrt
import random

In [2]:
# diccionario con coordenadas y si el area pertenece al protocolo 1 o no
area_data = {
    'A': [0,0,False],
    'EA1': [100,300, True],
    'EA2': [400,300,False],
    'EB1': [700,-300, True],
    'EB2': [400,-300,False],
    'EC1': [1000,200, True],
    'EC2': [1000,0,False],
}

### Ruta Optima Para Distribuir Los Productos

In [3]:
def getCostMatrix(areaList):
    graph = []
    for j in range(len(areaList)):
        graphRow = []
        for i in range(len(areaList)):
            p1 = area_data[areaList[j]]
            p2 = area_data[areaList[i]]
            d = getDistance(p1, p2)
            graphRow.append(d)
        graph.append(graphRow)
    return graph

In [4]:
def getDistance(p1, p2):
    return sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2) 

def pathIndexTranslate(areaList, pathIndexes):
    path = []
    for i in range(len(pathIndexes)):
        path.append(areaList[pathIndexes[i][1]])
    return path

In [5]:
def splitAreasByProtocol(areaList):
    proto_1_list = []
    proto_2_list = []

    for i in areaList:
        if area_data[i][2]:
            proto_1_list.append(i)
        else:
            proto_2_list.append(i)
    
    return proto_1_list, proto_2_list

In [6]:
def travellingBestRoute(graph, s, arealist):
 
    vertex = []
    for i in range(len(arealist)):
        if i != s:
            vertex.append(i)

    min_distance = maxsize
    min_path_i = []
    min_last_node_idx = None

    next_permutation=permutations(vertex)
    
    for i in next_permutation:
        current_pathweight = 0
        current_path_i = []

        k = s
        for j in i:
            current_pathweight += graph[k][j]
            current_path_i.append((k,j))
            k = j

        if current_pathweight < min_distance:
            min_distance = current_pathweight
            min_path_i = current_path_i
            min_last_node_idx = k
    
    # translate min path indexes to area name
    best_path = pathIndexTranslate(arealist, min_path_i)

    return min_distance, min_path_i, arealist[min_last_node_idx], best_path

In [7]:
def calculateBestPath(areaProto1List, areaProto2List, originNode):
    
    best_path = []
    total_distance = 0
    
    travel_proto1 = None
    
    # calculate best path for protocol 1 first
    if len(areaProto1List)>0:

        areaProto1List = originNode + areaProto1List
        
        areaProto1CostMatrix = getCostMatrix(areaProto1List)

        travel_proto1 = travellingBestRoute(areaProto1CostMatrix, 0, areaProto1List)
        
        best_path += travel_proto1[3]
        total_distance += travel_proto1[0]
    
    # calculate best path for protocol 2
    if len(areaProto2List) > 0:
        
        travel_proto2_ori = originNode
        
        if len(areaProto1List)>0:
            # set last node from protocol 1 path as origin node for protocol 2 path
            travel_proto2_ori = [travel_proto1[2]]
        
        areaProto2List = travel_proto2_ori + areaProto2List

        areaProto2CostMatrix = getCostMatrix(areaProto2List)

        travel_proto2 = travellingBestRoute(areaProto2CostMatrix, 0, areaProto2List)

        # adding best path and distance for protocol 2
        best_path += travel_proto2[3]
        total_distance += travel_proto2[0]

    return {'total_distance': total_distance, 'best_path': best_path}

In [8]:
 def GetBestRoute(areas_to_go):
    # set the origin
    area_origen = ['A']

    # split areas_to_go by protocol
    area_proto_1_list, aprea_proto_2_list = splitAreasByProtocol(areas_to_go)

    res = calculateBestPath(area_proto_1_list, aprea_proto_2_list, area_origen)
    
    return res

#### Ejemplo

In [9]:
print(GetBestRoute(['EA1','EA2', 'EB2','EC2', 'EB1', 'EC1']))

{'total_distance': 3218.6714861751616, 'best_path': ['EA1', 'EB1', 'EC1', 'EC2', 'EA2', 'EB2']}


### Importar Notebook con Modelo de Prediccion del Protocolo

In [10]:
%run Fase-2.4.b_LogisticRegression.ipynb

Matriz de Confusion: 
[[ 739    0]
 [   0 1261]]
Precision: 1.0, Recall: 1.0, F1: 1.0, Accuracy: 1.0




### Generador de Productos Aleatorios

In [11]:
def get_gaussian_value(min_num, max_num, media, std_dev):
    while True:
        val = random.gauss(media, std_dev)
        if val>=min_num and val<=max_num:
            return val

In [12]:
# Generador aleatorio de productos
def products_generator(n):
    products = []
    for i in range(n):
        
        embalaje = random.choice([0,1,2])
        largo = get_gaussian_value(5, 100, 50, 15)
        ancho = get_gaussian_value(5, 100, 50, 15)
        alto = get_gaussian_value(5, 100, 50, 15)
        peso = get_gaussian_value(1, 50, 25, 5)
        procedencia = random.choice([0,1,2,3])
        temperatura = random.choice([0,1])
        manipulacion = random.choice([0,1])
        
        products.append(np.array([embalaje, largo, ancho, alto, peso, procedencia, temperatura, manipulacion]))
        
    return products

### Logica del Vehiculo

In [13]:
# prediccion del protocolo usando un modelo de Logistic Regression 
def predict_protocol(p):
    y = LR_modelo_entrenado.predict(p.reshape(1,-1))
    proto = int(y[0])
    print("> Protocolo: ", proto)
    new_p = np.append (p, [proto]) # add protocol predicted to the product array
    return new_p

In [14]:
area_num_to_letter = {0:'A', 1:'B', 2:'C'}

# determina que areas visitar y cuantos productos entregar por area
def get_building_areas_to_go(products):
    
    dict_res = {}
    for p in products:
        
        area = area_num_to_letter[p[0]] # Letra del Area
        proto = int(p[-1]) # protocolo
        
        area_name = f"E{area}{proto}"
        
        if area_name not in dict_res:
            dict_res[area_name] = 1
        else:
            # aumenta el contador de producto para el area
            dict_res[area_name] += 1
            
    return dict_res

In [15]:
# usa la funcion GetBestRoute para determinar el orden en que debe recoger los productos
def get_bestroute(areas_visit):
    return GetBestRoute(areas_visit)

In [16]:
# ejecucion del vehiculo en un proceso iterativo
def vehicle_process(iterations):
    for i in range(iterations):
        print("=== Inicio del Recorrido ===")
        print("Recogiendo 3 productos del Almacen...")
        products = products_generator(3)

        
        # attach the protocol to each product
        print("Prediccion del protocolo de cada producto...")
        prods = [predict_protocol(p) for p in products]
        
        # get unique areas to visit and the product quantity to deliver
        print("Determinando cuantos productos entregar por cada area")
        areas_dic = get_building_areas_to_go(prods)
        print(areas_dic)
        
        print("Determinando la mejor ruta de entrega")
        best_route_res = get_bestroute(areas_dic.keys())
        print(best_route_res)
        print("=== Fin del Recorrido ===\n")
        

### Ejecucion del Vehiculo

In [17]:
# ejecucion del vehiculo 5 veces
vehicle_process(5)

=== Inicio del Recorrido ===
Recogiendo 3 productos del Almacen...
Prediccion del protocolo de cada producto...
> Protocolo:  1
> Protocolo:  1
> Protocolo:  2
Determinando cuantos productos entregar por cada area
{'EC1': 1, 'EB1': 1, 'EA2': 1}
Determinando la mejor ruta de entrega
{'total_distance': 1952.9487531007426, 'best_path': ['EB1', 'EC1', 'EA2']}
=== Fin del Recorrido ===

=== Inicio del Recorrido ===
Recogiendo 3 productos del Almacen...
Prediccion del protocolo de cada producto...
> Protocolo:  2
> Protocolo:  1
> Protocolo:  2
Determinando cuantos productos entregar por cada area
{'EB2': 2, 'EA1': 1}
Determinando la mejor ruta de entrega
{'total_distance': 987.048159266775, 'best_path': ['EA1', 'EB2']}
=== Fin del Recorrido ===

=== Inicio del Recorrido ===
Recogiendo 3 productos del Almacen...
Prediccion del protocolo de cada producto...
> Protocolo:  1
> Protocolo:  1
> Protocolo:  1
Determinando cuantos productos entregar por cada area
{'EC1': 1, 'EB1': 1, 'EA1': 1}
Dete